In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('./spam.csv', encoding='ISO-8859-1')

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data = df.to_numpy()

In [5]:
X = data[:, 1]
y = data[:, 0]

In [6]:
X.shape, y.shape

((5572,), (5572,))

In [7]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [8]:
def getStem(review):
  review = review.lower()
  tokens = tokenizer.tokenize(review)
  removed_stopwords = [w for w in tokens if w not in sw]
  stemmed_words = [ps.stem(token) for token in removed_stopwords]
  clean_review = ' '.join(stemmed_words)
  return clean_review

In [9]:
def getDoc(document):
  d = []
  for doc in document:
    d.append(getStem(doc))
  return d

In [10]:
stemmed_doc = getDoc(X)

In [11]:
stemmed_doc[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send å 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea å 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030']

In [12]:
cv = CountVectorizer()

In [13]:
vc = cv.fit_transform(stemmed_doc)

In [14]:
X = vc.todense()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [16]:
from sklearn.naive_bayes import MultinomialNB

In [17]:
model = MultinomialNB()

In [18]:
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
model.score(X_test, y_test)

0.977705274605764

In [30]:
messages = [
    """
    Hey there!

    In today's day, movies and TV shows are the first things that come to mind when you hear the the word 'Spin-off'. 
    But did you know it has a different meaning in the Stock Market? Find out what they are and who they benefit!
    Watch our engaging and easy-to-understand video from the #LearnWithUpstox series and find an answer to these and many more questions.
    We promise, no technical terms, only simple explanations!
    """,
    """Join us today at 12:00 PM ET / 16:00 UTC for a Red Hat DevNation tech talk on AWS Lambda and serverless Java with Bill Burke.
Have you ever tried Java on AWS Lambda but found that the cold-start latency and memory usage were far too high? 
In this session, we will show how we optimized Java for serverless applications by leveraging GraalVM with Quarkus to 
provide both supersonic startup speed and a subatomic memory footprint.""",

    """
    You left an item in your cart
    Hi, you added an item to your shopping cart and haven't completed your purchase. You can complete it now while it's still available.
"""
]

In [31]:
def prepare(messages):
  d = getDoc(messages)
  return cv.transform(d)

In [32]:
messages = prepare(messages)

In [33]:
y_pred = model.predict(messages)

In [34]:
y_pred

array(['ham', 'spam', 'ham'], dtype='<U4')